# Sequence to Sequence model to extract summary,objective from resume (input: resume line by line)


## 50 resumes


## Example 1:
### Input: 

### output:

In [6]:
from __future__ import print_function
import csv,os
import collections
from keras.models import load_model,Sequential
from keras.layers import Input,LSTM,Dense,Activation
from keras.callbacks import ModelCheckpoint
import numpy as np
batch_size=128
epochs=3
latent_dim=128
data_path="/home/santhosh/resumes_folder/custom_annotator/annotator-server/static/files/Data_Tracter_Resumes_in_TXT/csv"

## vectorizing the data

In [7]:
input_resumes=[]
output_summary=[]
input_tokens=collections.Counter()
output_tokens=set()
files=os.listdir(data_path)
files.sort()
count=0
for file in files[:50]:
    with open(data_path+'/'+file,'r') as csv_file:
        reader=csv.reader(csv_file)
        count=0
        for Input_text,output_text in reader:
            Input_text=Input_text.strip().lower()
            Input_text=Input_text
            
            input_resumes.append(Input_text)
            output_summary.append(output_text)
            
            for word in Input_text.split():
                if word not in input_tokens:
                    input_tokens[word]+=1
            for word in output_text.split():
                if word not in output_tokens:
                    output_tokens.add(word)
            if count==50:
                break
            count+=1


In [8]:
print(len(input_tokens))
num_encoder_tokens=min(len(input_tokens),4000)
input_tokens=[word for word,count in input_tokens.most_common(num_encoder_tokens-1)]
input_tokens=sorted(list(input_tokens))
output_tokens=sorted(list(output_tokens))
num_decoder_tokens=len(output_tokens)
max_encoder_seq_len=max([len(text.split()) for text in input_resumes])
max_decoder_seq_len=max([len(text.split()) for text in output_summary])

print('Number of samples:',len(input_resumes))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


5096
Number of samples: 2547
number of unique input token: 4000
number of unique output token: 2
Max Sequence length for inputs: 26
Max Sequence length for outputs: 1


## defining token2index

In [9]:
input_token2index=dict([(word,i) for i,word in enumerate(input_tokens)])
output_token2index=dict([(word,i) for i,word in enumerate(output_tokens)])
output_index2token=dict([(i,word) for i,word in enumerate(output_tokens)])
input_token2index['UNK']=num_encoder_tokens-1

In [10]:
output_index2token

{0: '0', 1: '1'}

## defing encoder_input,decoder_input and decoder_output

In [11]:
encoder_input_data=np.zeros((len(input_resumes),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_resumes),num_decoder_tokens),dtype='float32')


## creating training dataset

In [12]:
for i,(input_text,target_text) in enumerate(zip(input_resumes,output_summary)):
    for t,word in enumerate(input_text.split()[:max_encoder_seq_len]):
        if word not in input_token2index:
            word="UNK"
        encoder_input_data[i,t,input_token2index[word]]=1
    decoder_target_data[i,output_token2index[target_text]]=1
        

In [8]:
model = Sequential()
model.add(LSTM(latent_dim, return_sequences=False,
input_shape=(max_encoder_seq_len,num_encoder_tokens,)))
model.add(Dense(num_decoder_tokens))
model.add(Activation("softmax"))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               2114048   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 2,114,306
Trainable params: 2,114,306
Non-trainable params: 0
_________________________________________________________________


## function to test model

## Run training

In [ ]:
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/iteration_resume_line_classification.txt"
iteration=0

try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    # load weights
    print('loading the weights')
    file_path="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/model/resume_line_classification_iteration_"+str(iteration)+".h5"
    model=load_model(file_path)
    # estimate accuracy on whole dataset using loaded weights
    scores = model.evaluate(encoder_input_data,decoder_target_data,verbose=0)
    print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
    print("Testing Samples\n"+"-"*50)
    
    #Test model:
    print("-"*50)
    index=int(np.random.randint(len(input_resumes)/40*0.8)*40)
    test_input=""
    test_output=""
    for i in range(50):
        
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=model.predict(encoded_input_sequence, verbose=0)[0]
        output_sequence = output_index2token[np.argmax(output_sequence)]
        test_input+=input_resumes[index]+'\n'
        if output_sequence=="1":
            output_sequence=input_resumes[index]+'\n'
        else:
            output_sequence=''
        test_output+=output_sequence
        index+=1
    print("-"*50)
    print(test_input)
    print("---OUTPUT-----")
    print(test_output)
    print(" "*50+"-"*50)    

except:
    print('no file exist')


# checkpoint
filepath="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/checkpoints/resume_line_classification_checkpoints.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    try:
        print('Iteration:',iteration+1)
        #training
        model.fit(encoder_input_data,decoder_target_data ,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,callbacks=callbacks_list)

        #prepare sample_data to test 5 samples:
        print("-"*50)
        index=int(np.random.randint(len(input_resumes)/40*0.8)*40)
        test_input=""
        test_output=""
        for i in range(50):

            encoded_input_sequence=encoder_input_data[index: index + 1]
            output_sequence=model.predict(encoded_input_sequence, verbose=0)[0]
            output_sequence = output_index2token[np.argmax(output_sequence)]
            test_input+=input_resumes[index]+'\n'
            if output_sequence=="1":
                output_sequence=input_resumes[index]+'\n'
            else:
                output_sequence=''
            test_output+=output_sequence
            index+=1
        print("-"*50)
        print(test_input)
        print("---OUTPUT-----")
        print(test_output)
        print(" "*50+"-"*50)    
        # Save model
        file=open(iteration_file,'a')
        file.write('iteration:'+str(iteration+1)+'\n')
        file.close()
        iteration+=1
        file_path="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/model/resume_line_classification_iteration_"+str(iteration)+".h5"
        model.save(file_path)
    except:
        print("error in iteration",iteration)
        file=open(iteration_file,'r')
        last_line=file.read().split('\n')[-2]
        print('file_data,',last_line)
        iteration=int(last_line.split(':')[1])
        #print(iteration)
        file.close()
        # load weights
        print('loading the weights')
        file_path="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/model/resume_line_classification_iteration_"+str(iteration)+".h5"
        model=load_model(file_path)
        # estimate accuracy on whole dataset using loaded weights
        scores = model.evaluate(encoder_input_data,decoder_target_data,verbose=0)
        print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
        print("Testing Samples\n"+"-"*50)

        #Test model:
        print("-"*50)
        index=int(np.random.randint(len(input_resumes)/40*0.8)*40)
        test_input=""
        test_output=""
        for i in range(50):

            encoded_input_sequence=encoder_input_data[index: index + 1]
            output_sequence=model.predict(encoded_input_sequence, verbose=0)[0]
            output_sequence = output_index2token[np.argmax(output_sequence)]
            test_input+=input_resumes[index]+'\n'
            if output_sequence=="1":
                output_sequence=input_resumes[index]+'\n'
            else:
                output_sequence=''
            test_output+=output_sequence
            index+=1
        print("-"*50)
        print(test_input)
        print("---OUTPUT-----")
        print(test_output)
        print(" "*50+"-"*50)    

        

file_data, iteration:10
loading the weights
acc: 75.27%


Testing Samples
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
company tavant technologies
environment java, jquery, spring(mvc), hibernate, mysql, tomcat 6.0.
r
(+91) 9823976922
: abhay.chadha48@gmail.com
to work in learning and challenging environment, utilizing my skill and knowledge to the best of my abilities and
contribute positively to my personal growth as well as growth of the organization
experience summary
 4 years of overall software development experience
 hands-on experience in user interface and backend development of enterprise systems
 expertise in developing web applications using object oriented javascript and frameworks like
backbone.js, extensively worked on front-end technologies like require.js, underscore.js, jquery, ajax,
html5, compass, css3/sass, restful web services
 hands-on experience in cl

2037/2037 [==============================] - 24s 12ms/step - loss: 0.2778 - acc: 0.9013 - val_loss: 0.4428 - val_acc: 0.8353
Epoch 2/3
2037/2037 [==============================] - 26s 13ms/step - loss: 0.2160 - acc: 0.9190 - val_loss: 0.4662 - val_acc: 0.8275
Epoch 3/3
2037/2037 [==============================] - 26s 13ms/step - loss: 0.1663 - acc: 0.9391 - val_loss: 0.4183 - val_acc: 0.8451
--------------------------------------------------
--------------------------------------------------
form of notifications and to complete the tasks the user will have to choose from the
list of applications as listed in the workspace to connect to the respective apps and
complete the workflows required for completing the tasks. the user of this

page 1 of 6
ab h i j i t h b m
email: abhijithbm.1418@gmail.com mobile: +91-0-
99643 51418
abhijith has overall 6 years of experience in the e-commerce
area. he has a bachelor of business management degree and
specialization in finance. conversant with wi

Iteration: 16
Train on 2037 samples, validate on 510 samples
Epoch 1/3
2037/2037 [==============================] - 26s 13ms/step - loss: 0.0496 - acc: 0.9838 - val_loss: 0.7430 - val_acc: 0.8176
Epoch 2/3
2037/2037 [==============================] - 25s 12ms/step - loss: 0.0401 - acc: 0.9892 - val_loss: 0.6728 - val_acc: 0.8333
Epoch 3/3
2037/2037 [==============================] - 30s 14ms/step - loss: 0.0496 - acc: 0.9828 - val_loss: 0.6740 - val_acc: 0.8196
--------------------------------------------------
--------------------------------------------------
across mumbai and delhi.
• design of lan architecture for each site type including for dc and dr sites.
• design of security architecture for dc and dr sites.
• validation of network topology by setting a mock lab and preparation of configuration template in lab; based
on site type classification.
kotak group – data center implementation
resume
name – abhijeet kudale.
contact no – +91-9960819797
email – abhijeet.kudale567@gmail.

2037/2037 [==============================] - 27s 13ms/step - loss: 0.0433 - acc: 0.9858 - val_loss: 0.6839 - val_acc: 0.8275
Epoch 2/3
2037/2037 [==============================] - 28s 14ms/step - loss: 0.0296 - acc: 0.9907 - val_loss: 0.9378 - val_acc: 0.8196
Epoch 3/3
2037/2037 [==============================] - 27s 13ms/step - loss: 0.0270 - acc: 0.9907 - val_loss: 1.1236 - val_acc: 0.8176
--------------------------------------------------
--------------------------------------------------
 developed complex sql queries using dynamic sql, analytical functions and aggregate
functions.
 experience in sql performance tuning.
 used bulk collect, forall, save exception and sql%bulk_exception in pl/sql to
improve performance.
 worked on the table partitions like range partition and interval partition.
 knowledge in unix/linux.
 good analytical & troubleshooting skills and having good experience in customer interaction.
 good communication, presentation skills and an enthusiastic tea

2037/2037 [==============================] - 27s 13ms/step - loss: 0.0243 - acc: 0.9926 - val_loss: 0.9427 - val_acc: 0.8078
Epoch 2/3
2037/2037 [==============================] - 28s 14ms/step - loss: 0.0220 - acc: 0.9931 - val_loss: 0.9908 - val_acc: 0.8078
Epoch 3/3
2037/2037 [==============================] - 26s 13ms/step - loss: 0.0216 - acc: 0.9926 - val_loss: 1.0565 - val_acc: 0.8020
--------------------------------------------------
--------------------------------------------------
• worked in gamification product heybadges.com
this appication is to gamify users in online courses, onine gaming, etc.
real time notification is also available.application is successfuly integrated to
openedx projects.technologies used in this application are django ,
angularjs, mysql, mongodb, etc.
• worked in https://admin.staging.influenster.com
this application is used to crawl websites as scheduled time and save as
structured data. solr is used as search engine.
• technologies worked on
curri

2037/2037 [==============================] - 26s 13ms/step - loss: 0.0211 - acc: 0.9936 - val_loss: 1.2096 - val_acc: 0.6980
Epoch 2/3
2037/2037 [==============================] - 23s 11ms/step - loss: 0.0336 - acc: 0.9892 - val_loss: 0.7746 - val_acc: 0.7961
Epoch 3/3
2037/2037 [==============================] - 22s 11ms/step - loss: 0.0241 - acc: 0.9912 - val_loss: 0.8757 - val_acc: 0.8157
--------------------------------------------------
--------------------------------------------------
operations analytics
education and certifications:
title of the degree college/university year of passing
bachelor of business management pes college of management 2009
puc (pcmc‟s) national college 2006
sslc sree saraswathi vidya mandira 2004
technical competencies:
operating systems : windows xp, windows 2000, windows 8
ms office : excel, access, powerpoint, visio
db tool & techniques : sql and knowledge about vba projects
page 2 of 6
professional experience
client service management (data scienc

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-21-89df7a84a184>", line 59, in <module>
    validation_split=0.2,callbacks=callbacks_list)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/training.py", line 1042, in fit
    validation_steps=validation_steps)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/training_arrays.py", line 187, in fit_loop
    ins_batch = slice_arrays(ins, batch_ids)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/utils/generic_utils.py", line 507, in slice_arrays
    return [None if x is None else x[start] for x in arrays]
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/utils/generic_utils.py", line 507, in <listcomp>
    return [None if x is None else x[start] for x in arrays]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/santhosh/an


KeyboardInterrupt

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py",

## Testing

In [12]:
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/iteration_resume_line_classification.txt"

file=open(iteration_file,'r')
last_line=file.read().split('\n')[-2]
print('file_data,',last_line)
iteration=int(last_line.split(':')[1])
#print(iteration)
file.close()

print('loading the weights')
file_path="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_classification/model/resume_line_classification_iteration_"+str(iteration)+".h5"
model=load_model(file_path)
# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate(encoder_input_data,decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)

#Test model:
print("-"*50)
index=int(np.random.randint(len(input_resumes)/40*0.8)*40)
test_input=""
test_output=""
for i in range(50):

    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=model.predict(encoded_input_sequence, verbose=0)[0]
    output_sequence = output_index2token[np.argmax(output_sequence)]
    test_input+=input_resumes[index]+'\n'
    if output_sequence=="1":
        output_sequence=input_resumes[index]+'\n'
    else:
        output_sequence=''
    test_output+=output_sequence
    index+=1
print("-"*50)
print(test_input)
print("---OUTPUT-----")
print(test_output)
print(" "*50+"-"*50)    


file_data, iteration:10
loading the weights
acc: 75.27%


Testing Samples
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
a. ashokkumar, email: aashok411@gmail.com,
no#2, 6th cross, mobile: 9965623785
ramamurthy nagar,
bangalore.
profile summary:
• having 1 year 2+months of it experience in the field of developing software application using
technologies like html5, php, javascript, node.js, bootstrap and css.
• involved in user interface development using javascript, html5 and css,
• knowledge of payment gateway integration.
• mysql data integration with web applications.
• capability to develop efficient websites to server communication, through xml / json.
• working knowledge in electron framework to create desktop application.
professional experience:
working as an associate software engineer in jollybee apps pvt ltd (nov 2016 to present).
project #1
project title : dynamic ecomm